In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

from MAPutils import determineDevice, TrainTestSplitDLO, validate ,training_loop_ES, plotConfusionMatrix, training_loop_ES_with_soft_voting

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)



In [2]:
#device = determineDevice()
device = 'cpu'
print('Using device:', device)

Using device: cpu


In [3]:
transform = transforms.Compose(
    [
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [4]:
trainingSet, testSet = TrainTestSplitDLO(
                            baseDir = 'data/output_crawl_resized_train',
                            trainSize = 0.2,
                            transforms = transform,#augmentation,
                            random_state=42,
                            shuffle = True,
                            stratify = True
                        )       

In [5]:
class NetFinal(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.conv1_bn = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 64, kernel_size=3)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(32)
        self.dropout0 = nn.Dropout(p=0.7)
        self.fc0 = nn.Linear(32 * 28 * 51, 128)
        self.fc0_bn = nn.BatchNorm1d(32 * 28 * 51, 128)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, 3)

    def forward(self, x):
        x = F.max_pool2d(F.elu(self.conv1_bn(self.conv1(x))),2)
        x = F.max_pool2d(F.elu(self.conv2_bn(self.conv2(x))),2)
        x = F.max_pool2d(F.elu(self.conv3_bn(self.conv3(x))),2)
        #print(x.size()) # Debugging Line
        x = x.view(-1, 32 * 28 * 51)
        x = F.elu(self.fc0(self.dropout0(self.fc0_bn(x))))
        x = F.elu(self.fc1(self.dropout1(x)))
        x = F.elu(self.fc2(x))
        return x

In [6]:
import copy
import datetime 
import time
from tqdm.auto import tqdm

In [ ]:
train_loader = torch.utils.data.DataLoader(trainingSet, batch_size=64, shuffle=True)  
val_loader = torch.utils.data.DataLoader(testSet, batch_size=64, shuffle=False)
#train_loader = train_loader.to(device)

model = NetFinal()  
model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-4)  
loss_fn = nn.CrossEntropyLoss() 

batch_loss, epoch_loss = training_loop_ES(  
    n_epochs = 5000,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    val_loader = val_loader,
    EShistory = 10,
    device = device
)
model.eval()

In [7]:
# LOAD
model = NetFinal()
model.load_state_dict(torch.load('data/MAP_FinalModel.pth'))

<All keys matched successfully>

In [ ]:
plotConfusionMatrix(trainingSet, testSet, model)

In [ ]:
train_loader = torch.utils.data.DataLoader(trainingSet, batch_size=16, shuffle=False)
val_loader = torch.utils.data.DataLoader(testSet, batch_size=16, shuffle=False)

validate(model, train_loader, val_loader, device = device)

In [ ]:
# SAVE
PATH = 'data/MAP_benchmark_Final.pth'
torch.save(model.state_dict(), PATH)

In [9]:
test_data_dir = 'data/output_crawl_resized_test'

model.eval()s

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = ImageFolder(test_data_dir, transform=transform)

correct = 0
total = 0

with torch.no_grad():
    for i in range(len(test_dataset)):
        image, label = test_dataset[i]
        image = image.unsqueeze(0)
        output = model(image)
        _, predicted = torch.max(output.data, 1)
        total += 1
        correct += (predicted == label).sum().item()

accuracy = (correct / total) * 100
print(f'Genauigkeit auf den Testbildern: {accuracy:.2f}%')

Genauigkeit auf den Testbildern: 83.23%
